In [99]:
# Install dependencies as needed:

import kagglehub

from kagglehub import KaggleDatasetAdapter

from datasets import load_dataset

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

import pandas as pd

import joblib

## MODELO EM PORTUGUÊS

In [100]:

# Set the path to the file you'd like to load
file_path = 'olist_order_reviews_dataset.csv'

# Load the latest version
df_olist = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "olistbr/brazilian-ecommerce", file_path, pandas_kwargs={"encoding":'utf-8', "quotechar":'"', "on_bad_lines":'skip'})

hf = load_dataset("ruanchaves/b2w-reviews01")

df_b2w = hf['train'].to_pandas()

In [101]:
df_olist.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [102]:
df_b2w.head()

,submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state
0,2018-01-01 00:11:28,d0fb1ca69422530334178f5c8624aa7a99da47907c44de...,132532965,Notebook Asus Vivobook Max X541NA-GO472T Intel...,None,Informática,Notebook,Bom,4,Yes,Estou contente com a compra entrega rápida o ú...,1958.0,F,RJ
1,2018-01-01 00:13:48,014d6dc5a10aed1ff1e6f349fb2b059a2d3de511c7538a...,22562178,Copo Acrílico Com Canudo 500ml Rocie,None,Utilidades Domésticas,"Copos, Taças e Canecas","Preço imbatível, ótima qualidade",4,Yes,"Por apenas R$1994.20,eu consegui comprar esse ...",1996.0,M,SC
2,2018-01-01 00:26:02,44f2c8edd93471926fff601274b8b2b5c4824e386ae4f2...,113022329,Panela de Pressão Elétrica Philips Walita Dail...,philips walita,Eletroportáteis,Panela Elétrica,ATENDE TODAS AS EXPECTATIVA.,4,Yes,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1984.0,M,SP
3,2018-01-01 00:35:54,ce741665c1764ab2d77539e18d0e4f66dde6213c9f0863...,113851581,Betoneira Columbus - Roma Brinquedos,roma jensen,Brinquedos,Veículos de Brinquedo,presente mais que desejado,4,Yes,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1985.0,F,SP
4,2018-01-01 01:00:28,7d7b6b18dda804a897359276cef0ca252f9932bf4b5c8e...,131788803,"Smart TV LED 43"" LG 43UJ6525 Ultra HD 4K com C...",lg,TV e Home Theater,TV,"Sem duvidas, excelente",5,Yes,"A entrega foi no prazo, as americanas estão de...",1994.0,M,MG


In [103]:
# Tratamento, limpeza e padronização dos dados

df_b2w.columns


Index(['submission_date', 'reviewer_id', 'product_id', 'product_name',
       'product_brand', 'site_category_lv1', 'site_category_lv2',
       'review_title', 'overall_rating', 'recommend_to_a_friend',
       'review_text', 'reviewer_birth_year', 'reviewer_gender',
       'reviewer_state'],
      dtype='object')

In [104]:
df_b2w.drop(axis="columns", columns=(['submission_date', 'reviewer_id', 'product_id', 'product_name',
       'product_brand', 'site_category_lv1', 'site_category_lv2',
       'review_title', 'reviewer_birth_year', 'reviewer_gender', 'reviewer_state', 'recommend_to_a_friend']), inplace=True)

df_b2w.columns

Index(['overall_rating', 'review_text'], dtype='object')

In [105]:
df_b2w.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132373 entries, 0 to 132372
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   overall_rating  132373 non-null  int32 
 1   review_text     129098 non-null  object
dtypes: int32(1), object(1)
memory usage: 1.5+ MB


In [106]:
df_olist.columns

Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object')

In [107]:
df_olist.drop(axis="columns", columns=(['review_id', 'order_id','review_creation_date',
       'review_answer_timestamp', 'review_comment_title']), inplace=True)

df_olist.columns

Index(['review_score', 'review_comment_message'], dtype='object')

In [108]:
df_olist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   review_score            99224 non-null  int64 
 1   review_comment_message  40977 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.5+ MB


In [109]:
# TRATAR OS VALORES NULOS

df_b2w.dropna(subset=['review_text'], inplace=True)

df_olist.dropna(subset=['review_comment_message'], inplace=True)



In [110]:
df_b2w.info()

<class 'pandas.core.frame.DataFrame'>
Index: 129098 entries, 0 to 132372
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   overall_rating  129098 non-null  int32 
 1   review_text     129098 non-null  object
dtypes: int32(1), object(1)
memory usage: 2.5+ MB


In [111]:
df_olist.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40977 entries, 3 to 99223
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   review_score            40977 non-null  int64 
 1   review_comment_message  40977 non-null  object
dtypes: int64(1), object(1)
memory usage: 960.4+ KB


In [112]:
# "Converter" a avaliação de cada dataset para o padrão ["Negativo", "Neutro", "Positivo"]

df_b2w.overall_rating.unique()

array([4, 5, 1, 2, 3], dtype=int32)

In [113]:
df_olist.review_score.unique()

array([5, 4, 2, 1, 3])

In [114]:
# DF_B2B:
# 1,2 => Negativo - 3 => Neutro - 4,5 => Positivo. Criamos uma nova coluna com esse "Mapeamento" usando a função .map()

mapa_sentimentos = {
    1: 'Negativo',
    2: 'Negativo',
    3: 'Neutro',
    4: 'Positivo',
    5: 'Positivo'
}

df_b2w['sentimento'] = df_b2w['overall_rating'].map(mapa_sentimentos)
df_olist['sentimento'] = df_olist['review_score'].map(mapa_sentimentos)

In [115]:
df_b2w.head()

,overall_rating,review_text,sentimento
0,4,Estou contente com a compra entrega rápida o ú...,Positivo
1,4,"Por apenas R$1994.20,eu consegui comprar esse ...",Positivo
2,4,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,Positivo
3,4,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,Positivo
4,5,"A entrega foi no prazo, as americanas estão de...",Positivo


In [116]:
df_olist.head()

,review_score,review_comment_message,sentimento
3,5,Recebi bem antes do prazo estipulado.,Positivo
4,5,Parabéns lojas lannister adorei comprar pela I...,Positivo
9,4,aparelho eficiente. no site a marca do aparelh...,Positivo
12,4,"Mas um pouco ,travando...pelo valor ta Boa.\r\n",Positivo
15,5,"Vendedor confiável, produto ok e entrega antes...",Positivo


In [117]:
# podemos descartar agora as colunas review_score e overall_rating

df_olist.drop(axis="columns", columns=(['review_score']), inplace=True)

df_b2w.drop(axis="columns", columns=(['overall_rating']), inplace=True)


In [118]:
df_olist.columns

Index(['review_comment_message', 'sentimento'], dtype='object')

In [119]:
df_b2w.columns

Index(['review_text', 'sentimento'], dtype='object')

In [120]:
# agora, trocaremos os nomes das colunas "review_comment_message" e "review_text" por apenas "texto", para padronização

df_olist.columns = ['texto', 'sentimento']
df_b2w.columns = ['texto', 'sentimento']

In [121]:
df_b2w.columns

Index(['texto', 'sentimento'], dtype='object')

In [122]:
df_olist.columns

Index(['texto', 'sentimento'], dtype='object')

In [123]:
df_b2w.head()

,texto,sentimento
0,Estou contente com a compra entrega rápida o ú...,Positivo
1,"Por apenas R$1994.20,eu consegui comprar esse ...",Positivo
2,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,Positivo
3,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,Positivo
4,"A entrega foi no prazo, as americanas estão de...",Positivo


In [124]:
df_olist.head()

,texto,sentimento
3,Recebi bem antes do prazo estipulado.,Positivo
4,Parabéns lojas lannister adorei comprar pela I...,Positivo
9,aparelho eficiente. no site a marca do aparelh...,Positivo
12,"Mas um pouco ,travando...pelo valor ta Boa.\r\n",Positivo
15,"Vendedor confiável, produto ok e entrega antes...",Positivo


In [125]:
# Agora, concatenamos os dois datasets, ao mesmo tempo em que resetamos os index e os "embaralhamos"

# 1. Concatenar os dois dataframes (empilhar um sobre o outro)
# ignore_index=True ajuda a não duplicar índices antigos, mas o reset abaixo garante a limpeza final
df_final = pd.concat([df_olist, df_b2w], ignore_index=True)

# 2. Embaralhar as linhas
# frac=1 significa "retorne 100% dos dados", mas em ordem aleatória
# reset_index(drop=True) recria o índice de 0 a N, para não ficar tudo bagunçado (ex: 5, 100, 2...)
df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)

df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170075 entries, 0 to 170074
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   texto       170075 non-null  object
 1   sentimento  170075 non-null  object
dtypes: object(2)
memory usage: 2.6+ MB


In [126]:
# salvaremos este df_final em um arquivo .csv

df_final.to_csv('../datasets/df_pt_concatenado.csv', index=False, header=True, sep=';')


In [127]:
df_final.sentimento.unique()

array(['Neutro', 'Positivo', 'Negativo'], dtype=object)

In [128]:
df_final.describe()

,texto,sentimento
count,170075,170075
unique,162869,3
top,Muito bom,Positivo
freq,230,105846


In [129]:
# Tratamento do texto
# Agora, iremos tratar o texto para retirada de acentos, caracteres especiais, espaços em branco, etc.



In [130]:
# Pipeline de treinamento

modelo = make_pipeline(
    TfidfVectorizer(max_features=10000, strip_accents='unicode', lowercase=True, ngram_range=(1, 2)),
    LogisticRegression(solver='lbfgs', max_iter=1000)
)

print("⏳ Treinando modelo...")
modelo.fit(df_final['texto'], df_final['sentimento'])

# 3. Salvar
joblib.dump(modelo, "../models/modelo_pt.joblib")
print("✅ Novo modelo salvo em: modelo_pt.joblib")

⏳ Treinando modelo...
✅ Novo modelo salvo em: modelo_pt.joblib
